##### DB 연결(db_conn)

In [1]:
# DB Connection

import configparser
import pymysql

# config open
config = configparser.ConfigParser()
config.read('../config/config.ini')

class DBConnection:
    def __init__(self):
        self._conn = pymysql.connect(           
            # Local DB
            host = config['DB_TEST']['HOST'],
            port = 3306,
            user = config['DB_TEST']['USER'],
            password = config['DB_TEST']['PASSWD'],
            db = config['DB_TEST']['DB_NAME'],

            #autocommit = True,           
            charset = 'utf8mb4',
            cursorclass = pymysql.cursors.DictCursor # DB조회시 컬럼명을 동시에 보여줌
        )
        self._cursor = self._conn.cursor()
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    @property
    def connection(self):
        return self._conn
    
    @property
    def cursor(self):
        return self._cursor
    
    def rollback(self, rollback=True):
        if rollback:
            self.rollback()
        self.connection.close()
    
    def commit(self):
        self.connection.commit()
    
    def close(self, commit=True):
        if commit:
            self.commit()
        self.connection.close()
    
    def execute(self, sql, params=None):
        self.cursor.execute(sql, params or ())

    def executemany(self, sql, params=None):
        self.cursor.executemany(sql, params or ())

    def fetchall(self):
        return self.cursor.fetchall()
    
    def fetchone(self):
        return self.cursor.fetchone()
    
    def query(self, sql, params=None):
        self.cursor.execute(sql, params or ())
        return self.fetchall()
    
    def rows(self):
        return self.cursor.rowcount    

In [2]:
import pandas as pd

conn = DBConnection()

##### 카테고리(tb_category) 데이터 입력

In [3]:
path = '../assets/'
file_name = 'keyword_list.csv'

file_path = path + file_name

df = pd.read_csv(file_path, encoding='utf-8')

# 중복 제거
cate = df.drop_duplicates(['카테고리'])
cate_datas = list(cate['카테고리'].values)

# print(datas)

cate_sql = "INSERT INTO tb_category(cate_name)VALUES(%s)"

values = []
for cate_data in cate_datas:
    values.append(cate_data)

conn.executemany(cate_sql, values)
conn.commit()


##### 키워드 데이터 입력

In [4]:
path = '../assets/'
file_name = 'keyword_list.csv'

file_path = path + file_name

df = pd.read_csv(file_path, encoding='utf-8')

# 카테고리 명 추출
select_query = """
SELECT (SELECT cate_name FROM tb_category WHERE cate_code = 1) AS 'e_cate_name', # 환경
	(SELECT cate_name FROM tb_category WHERE cate_code = 2) AS 's_cate_name', # 사회
	(SELECT cate_name FROM tb_category WHERE cate_code = 3) AS 'g_cate_name' # 지배구조
"""
conn.execute(select_query)
cate_result = conn.fetchone()
e_cate_name = cate_result['e_cate_name'] # 환경(E)
s_cate_name = cate_result['s_cate_name'] # 사회(S)
g_cate_name = cate_result['g_cate_name'] # 지배구조(G)

# insert query문
keyword_sql = """INSERT INTO tb_keyword(cate_code, key_word)VALUES(%s, %s);"""

# 환경 키워드 insert
e_keyword = df[df['카테고리'] == e_cate_name]
e_datas = list(e_keyword['키워드'])

keyword_values = []
for e_data in e_datas:
    keyword_values.append((1, e_data))

# 사회 키워드 insert
s_keyword = df[df['카테고리'] == s_cate_name]
s_datas = list(s_keyword['키워드'])

for s_data in s_datas:
    keyword_values.append((2, s_data))

# 지배구조 키워드 insert
g_keyword = df[df['카테고리'] == g_cate_name]
g_datas = list(g_keyword['키워드'])

for g_data in g_datas:
    keyword_values.append((3, g_data))

# print(keyword_values)
conn.executemany(keyword_sql, keyword_values)

conn.commit()

##### 기업 목록 데이터 입력

In [5]:
path = '../assets/'
file_name = 'company_list.csv'

file_path = path + file_name

df = pd.read_csv(file_path, encoding='utf-8')

# df
# 기업 리스트
company_list = list(df['COM_ABBRV'])

company_query = """INSERT INTO tb_company(company_name)VALUES(%s);"""

company_params = []
for company in company_list:
    company_params.append(company)

conn.executemany(company_query, company_params)

conn.commit()

In [6]:
# DB 연결 객체 종료
conn.close()